<a href="https://colab.research.google.com/github/T-adnane/test/blob/master/Sign_language_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def mediapipe_landmarks(image, results):
        # Hands landmarks
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
        # Pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=1))

In [ ]:
#Extraire les Valeurs de Mediapipe sous forme de vecteurs
def extract_Values(results):
    # On a 33 landmark dans le pose est chaque landmark est dans l'espace alors 3paramétre et ona aussi la visibilité alors 4 paramétres alors en fin on a 33*4 paramétre
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # On a 21 land mark on chaque main et chaque landmark est représenté en espace alors ona 21*3 valeurs
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # retourner les 3 listes concaténées
    return np.concatenate([pose, lh, rh])

In [ ]:
def count_frames(video_path):
    # Ouvrir la vidéo en utilisant OpenCV
    video = cv2.VideoCapture(video_path)
    
    # Vérifier si la vidéo est ouverte avec succès
    if not video.isOpened():
        print("Impossible d'ouvrir la vidéo :", video_path)
        return
    
    # Obtenir le nombre total de frames dans la vidéo
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Fermer la vidéo
    video.release()
    
    return total_frames

In [4]:
import cv2
import mediapipe as mp
import os
import numpy as np

In [16]:

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  # Specify the path to the directory containing the videos
  directory_path = r"/content/drive/MyDrive/Adnane_Mois_Keypoints/janvier"
  save_dir = r"/content/drive/MyDrive/Adnane_Mois_Keypoints/janvier_keypoints"
  for alpha in range(50,150,2):
    # Loop through all video files in the directory
    for filename in os.listdir(directory_path):
      if filename.endswith(".mp4"):
        # Read video
        video_path = os.path.join(directory_path, filename)
        num_frames = count_frames(video_path)
        cap = cv2.VideoCapture(video_path)
        keypoints_list = []
        nbr,k = 0,0
        save_frames = int(num_frames/30)
        while nbr <= 30:
          # Read next frame
          ret, frame = cap.read()
          # Check if end of video
          if not ret:
            keypoints_list.append([0] * 258)
            nbr+=1
          else:
            height, width = frame.shape[:2]
            width1 = int(width * alpha / 100)
            height1 = int(height * alpha / 100)
            dim = (width1, height1)
            # redimentioner le frame
            frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            if k % save_frames == 0:
              # Extract keypoints
              keypoints = extract_Values(results)
              keypoints_list.append(keypoints)
              nbr += 1
            k += 1
        # Save keypoints to a file
        npy_path = os.path.join(save_dir, filename.split(".")[0] + str(alpha-50) + ".npy")
        np.save(npy_path, keypoints_list)

## **Modèle**

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
DATA_PATH = '/content/drive/MyDrive/Adnane_Mois_Keypoints/KeyPoints'
signs = np.array(['janvier', 'fevrier'])
no_sequences = 250
label_map = {label:num for num, label in enumerate(signs)}

In [6]:
label_map

{'janvier': 0, 'fevrier': 1}

In [7]:
sequences, labels = [], []
for sign in signs:
    for filename in os.listdir(os.path.join(DATA_PATH, sign)):
        res = np.load(os.path.join(DATA_PATH, sign,filename))
        sequences.append(res)
        labels.append(label_map[sign])

In [8]:
res.shape

(31, 258)

In [9]:
X = np.array(sequences)

In [10]:
y = to_categorical(labels).astype(int)

In [11]:
y.shape

(500, 2)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
X_train.shape

(350, 31, 258)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [15]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [16]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(31,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))

In [17]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
11/11 [==============================] - 5s 74ms/step - loss: 0.7468 - categorical_accuracy: 0.5514
Epoch 2/100
11/11 [==============================] - 1s 70ms/step - loss: 0.6620 - categorical_accuracy: 0.6314
Epoch 3/100
11/11 [==============================] - 1s 67ms/step - loss: 0.6092 - categorical_accuracy: 0.5943
Epoch 4/100
11/11 [==============================] - 1s 67ms/step - loss: 0.5387 - categorical_accuracy: 0.6057
Epoch 5/100
11/11 [==============================] - 1s 69ms/step - loss: 1.1947 - categorical_accuracy: 0.6486
Epoch 6/100
11/11 [==============================] - 1s 109ms/step - loss: 0.6249 - categorical_accuracy: 0.6657
Epoch 7/100
11/11 [==============================] - 1s 111ms/step - loss: 0.6858 - categorical_accuracy: 0.6771
Epoch 8/100
11/11 [==============================] - 1s 114ms/step - loss: 0.4580 - categorical_accuracy: 0.7743
Epoch 9/100
11/11 [==============================] - 1s 70ms/step - loss: 0.3547 - categorical_accura

In [19]:
model.save('janFev.h5')

In [ ]:
# import cv2
# import mediapipe as mp
# import os
# import numpy as np
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     # Specify the path to the directory containing the videos
#     directory_path = r"/content/avril"
#     save_dir = r"/content/avril06"
    
#     for angle in range(50):
#         scale_percent = 2*angle+50
#         # Loop through all video files in the directory
#         for filename in os.listdir(directory_path):
#             if filename.endswith(".mp4"):
#                 # Read video
#                 video_path = os.path.join(directory_path, filename)
#                 cap = cv2.VideoCapture(video_path)
#                 keypoints_list = []
#                 nbr = 0
              
#                 while nbr < 30:
#                     # Read next frame
#                     ret, frame = cap.read()
#                     # Check if end of video
#                     if not ret:
#                       keypoints_list.append([0] * 258)
#                       nbr+=1
#                     else:
#                       c = frame.shape[:2]
#                       height, width = c
#                       width1 = int(width * scale_percent / 100)
#                       height1 = int(height * scale_percent / 100)
#                       dim = (width1, height1)
#                       # Calculate the rotation matrix
#                       #rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
#                       # Rotate and redimensionner the frame
#                       #frame = cv2.warpAffine(frame, rotation_matrix, (width, height))
#                       frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
#                       # Make detections
#                       image, results = mediapipe_detection(frame, holistic)

#                       # Draw landmarks
#                       #mediapipe_landmarks(image, results)

#                       # Show to screen
#                       #cv2.imshow('OpenCV Feed', image)

#                       # Extract keypoints
                      
#                       keypoints = extract_Values(results)
#                       keypoints_list.append(keypoints)

#                       # Break gracefully
#                       if cv2.waitKey(10) & 0xFF == ord('q'):
#                           break
#                       nbr += 1
#                 # Save keypoints to a file
#                 npy_path = os.path.join(save_dir, filename.split(".")[0] + str(angle) + ".npy")
#                 np.save(npy_path, keypoints_list)

#                 cap.release()
#                 cv2.destroyAllWindows()

In [ ]:
donnees = np.load("/content/fevrier_keyPoints/fevrier60.npy")

In [ ]:
donnees.shape

(60, 258)

In [ ]:
donnees.shape

(80, 258)

In [ ]:
donnees.shape

(95, 258)

In [ ]:
donnees.shape

(60, 258)

In [ ]:
donnees.shape

(44, 258)

In [ ]:
donnees.shape

(73, 258)

In [ ]:
import cv2

def count_frames(video_path):
    # Ouvrir la vidéo en utilisant OpenCV
    video = cv2.VideoCapture(video_path)
    
    # Vérifier si la vidéo est ouverte avec succès
    if not video.isOpened():
        print("Impossible d'ouvrir la vidéo :", video_path)
        return
    
    # Obtenir le nombre total de frames dans la vidéo
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Fermer la vidéo
    video.release()
    
    return total_frames

# Chemin vers la vidéo
video_path = "/content/janv/janvier6.mp4"

# Appeler la fonction pour obtenir le nombre de frames
num_frames = count_frames(video_path)

# Afficher le résultat
print("Le nombre de frames dans la vidéo est :", num_frames)


Le nombre de frames dans la vidéo est : 31
